In [1]:
# ====================================================================================
#  seleccionar el corte deseado (ver archivo de densidades) y cambiar el numero de plot
# ====================================================================================
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rc
import os
from cuts import density_data
import plot
import HOD

rc('text', usetex=True)
rc('font', family='serif')

In [2]:
for model in ['G13']:
    for cut in ['Mstell', 'SFR']:
        for d in [1,2,3,4]:
    
            plot.HOD(model, d, cut, Mock=True, cSAM = True, cShuffle=True)

In [ ]:
model = 'G13_millennium'
path = "/home/esteban/Escritorio/Practica/Data/" + model
log_centralmvir = np.log10(np.load(path + '/centralmvir.npy') + 1e-10) + 10
type_gal = np.load(path + '/type.npy')

In [ ]:
%%time
type_cuts = ['Mstell', 'SFR']
number_densities = [1,2,3,4]
NBIN = 30
HOD.HOD(NBIN, type_cuts, number_densities, model, log_halomass=log_centralmvir, type_gal=type_gal, SAM=False, Shuffle=True)

In [ ]:
ab = str(raw_input("Model: "))
model = ab + "_millennium"
type_cut = int(raw_input("Select criteria:\n Stellarmass cuts: 1\n SFR cuts: 2\n Choice:"))

In [ ]:
path = "/home/esteban/Escritorio/Practica/Data/" + model
os.chdir( path )

In [ ]:
#import cuts
sfr = np.load('sfr.npy')
stellarmass = np.load('stellarmass.npy')
if ab == 'G13': log_centralmvir = np.log10(np.load('centralmvir.npy') + 1e-10) + 10 # MPA
else: log_centralmvir = np.log10(np.load('mdhalo.npy') + 1e-10) + 10  # Durham
Type = np.load('type.npy')

#Density samples (Only 1 run!) Done!
#cuts.density_data(sfr, stellarmass)

In [ ]:
logm_min = 10
logm_max = 15.
NBIN = 30

mass_labels = ((log_centralmvir - logm_min)/(logm_max - logm_min) * NBIN).astype(int)
bin_width = (logm_max - logm_min)/NBIN
bins = np.array([logm_min + bin_width*i for i in range(NBIN)]) + bin_width *0.5

In [ ]:
%%time
#n = [20]
n = [5, 10, 15, 20, 25]
#n = [1,2,3]
log_rho, mass_cuts, sfr_cuts = np.loadtxt('density_data.txt', unpack = True)

for k in n:
    #path = "/home/esteban/Escritorio/Practica/Data/" + model
    #os.chdir( path )
    
    # Density selection
    # =================
    #   Fila    log(rho)
    #    25      -1.5
    #    20      -2.0
    #    15      -2.5
    #    10      -3.0
    #     5      -3.5
    # =================

    #log_rho, mass_cuts, sfr_cuts = np.loadtxt('density_data.txt', unpack = True)
    rho, mass_cut, sfr_cut = log_rho[k], mass_cuts[k], sfr_cuts[k]

    if type_cut == 1: feature = stellarmass; cut = mass_cut ; tc = 'Stellar Mass/'
    else: feature = sfr; cut = sfr_cut; tc = 'SFR/'

    # To make HODs
    HOD = np.zeros(NBIN)
    Cent = np.zeros(NBIN)
    Sats = np.zeros(NBIN)

    for i in range(NBIN):

        idx = np.where(mass_labels == i)[0]
        Halos = np.where(Type[idx] == 0)[0] 

        mask_HOD = feature[idx] > cut # Msun/year
        HOD_ID = np.where(mask_HOD)[0]

        mask_Cent = np.where(Type[idx][HOD_ID] == 0)[0]
        mask_Sats = np.where(Type[idx][HOD_ID] != 0)[0]

        Len_Full = len(HOD_ID)
        Len_Cent = len(mask_Cent)
        Len_Sats = len(mask_Sats)
        Len_Haloes = float(len(Halos)) + 1e-10

        HOD[i]      = Len_Full / Len_Haloes 
        Cent[i]     = Len_Cent / Len_Haloes
        Sats[i]     = Len_Sats / Len_Haloes
        
    HOD = np.log10(HOD + 1e-10)
    Cent = np.log10(Cent + 1e-10)
    Sats = np.log10(Sats + 1e-10)

    #mm = path + '/hod_data/' + tc 
    #np.savetxt(mm + 'hod_%.1f_15b' % abs(rho), np.array([bins, HOD, Cent, Sats]).T)
    f = plt.figure()
    plt.plot(bins, HOD, 'k.', label = 'All')
    plt.plot(bins, Cent, 'r--', label = 'Cent')
    plt.plot(bins, Sats, 'b--', label = 'Sats')
    plt.axhline(y = 0, color='black', linestyle='--', lw=.4)
    plt.title(r"$n =10^{-%.1f}/h^{-3} Mpc^3$" %rho)
    plt.xlabel(r"$log(M_h / h^{-1}M_{\odot})$", fontsize = 10)
    plt.ylabel(r"$log(N)$", fontsize = 10)
    plt.ylim(-6,3)
    plt.xlim(10,15)
    plt.minorticks_on()
    plt.legend(loc='upper left',title = r"\textbf{%s}"% ab, frameon = False, prop={'size':10})
    f.savefig(path + '/hod_%.1f_zoomoff.pdf' %abs(rho))           
               


In [ ]:
# ======== TO PLOT ===========
f = plt.figure()
#path = "/home/esteban/Escritorio/Practica/Plots/" + model + '/HOD/' + tc
path = "/home/esteban/Escritorio/Practica/Data/" + model + '/hod_data/' + tc
os.chdir( path )
rho = 2.0

bins1, HOD1, Cent1, Sats1 = np.loadtxt(mm + 'hod_%.1f' %rho, unpack=True)
bins2, HOD2, Cent2, Sats2 = np.loadtxt(mm + 'hod_%.1f_15b' %rho, unpack=True)
bins3, HOD3, Cent3, Sats3 = np.loadtxt(mm + 'hod_%.1f_60b' %rho, unpack=True)

plt.plot(bins1, HOD1, 'k-', lw = .7, label = 'All (30 Bins)')
plt.plot(bins2, HOD2, 'r-', lw = .7, label = 'All (15 Bins)')
plt.plot(bins3, HOD3, 'b-', lw = .7, label = 'All (60 Bins)')
plt.axhline(y = 0, color='black', linestyle='--', lw=.4)
#plt.plot(bins, Cent, 'r--', label = 'Cent')
#plt.plot(bins, Sats, 'b--', label = 'Sats')
#plt.plot(bins, line, 'k--', lw = .5 )
plt.title(r"$n = 10^{-%.1f} /h^{-3} Mpc^3$" %rho)
plt.xlabel(r"$log(M_h / h^{-1}M_{\odot})$", fontsize = 10)
plt.ylabel(r"$log(N)$", fontsize = 10)
plt.ylim(-6,3)
plt.xlim(10,15)
plt.minorticks_on()
plt.legend(loc='upper left',title = r"\textbf{%s}"% ab, frameon = False, prop={'size':10})
f.savefig('hod_%.1f.pdf' %rho)